In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
import os

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18,googlenet,densenet121
from sklearn.model_selection import train_test_split
from PIL import Image

In [22]:
def multiple_dir_files_to_df(dirs, label):
    df = pd.DataFrame()
    for idx,dir in enumerate(dirs):
        files = os.listdir(dir)
        files = [dir + '/' + file for file in files]
        df = pd.concat([df,pd.DataFrame({'file': files, 'label': label[idx]})])
    return df

common_path="Database/"

labels=["neutral","happy","sad","fear","angry","surprise","disgust","Happily_Surprised",
"Happily_Disgusted","Sadly_Fearful","Sadly_Angry","Sadly_Surprised","Sadly_Disgusted",
"Fearfully_Angry","Fearfully_Surprised","Fearfully_Disgusted","Angrily_Surprised",
"Angrily_Disgusted","Disgustedly_Surprised","Appalled","Hatred","Awed"]

database=[common_path+label for label in labels]

df = multiple_dir_files_to_df(database,np.linspace(0,len(labels)-1,len(labels)))
X,y=df["file"].tolist(),df["label"].tolist()

In [23]:
# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, file_names, labels, transform=None):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = f'{self.file_names[idx]}' 
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [24]:
# Define the image transformations
transform = transforms.Compose([
    transforms.CenterCrop((500, 500)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [25]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.googlenet = googlenet(pretrained=True)
        self.googlenet.fc = nn.Linear(512, 7)
        self.googlenet.inception4d=nn.Identity()
        self.googlenet.inception4e=nn.Identity()
        self.googlenet.inception5a=nn.Identity()
        self.googlenet.inception5b=nn.Identity()
        self.googlenet.Linear=nn.Identity()

    def forward(self, x):
        return self.googlenet(x)

model= GoogleNet()
print(model)

GoogleNet(
  (googlenet): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
# Définir le modèle ResNet
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(256, 7)
        self.resnet.layer4= nn.Identity()
        self.resnet.layer1[-1].bn2 = nn.Sequential(
            self.resnet.layer1[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer2[-1].bn2 = nn.Sequential(
            self.resnet.layer2[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer3[-1].bn2 = nn.Sequential(
            self.resnet.layer3[-1].bn2,
            nn.Dropout2d(p=0.15)
        )
        self.resnet.fc=nn.Identity()
        
    def forward(self, x):
        return self.resnet(x)

model= ResNetModel()
print(model)

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [27]:
class DenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet = densenet121(pretrained=True)
        self.densenet.classifier = nn.Identity()

    def forward(self, x):
        return self.densenet(x)
    
print(DenseNet())

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0

In [28]:
# Créer les jeux de données et les chargeurs de données
t_dataset = CustomDataset(X, y, transform)
t_loader = DataLoader(t_dataset, batch_size=10, shuffle=False)

In [29]:
#Features extraction
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model ="./densenet_pretrained_7_emo_18.pth"
model = DenseNet()  
model.load_state_dict(torch.load(pretrained_model, map_location=device,strict=False))
model.eval()
df=pd.DataFrame()

for i, (x, y) in enumerate(t_loader):
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        output = model(x)
        df=pd.concat([df,pd.DataFrame(output.cpu().numpy())])
        print(df.shape)

(10, 1024)
(20, 1024)
(30, 1024)
(40, 1024)
(50, 1024)
(60, 1024)
(70, 1024)
(80, 1024)
(90, 1024)
(100, 1024)
(110, 1024)
(120, 1024)
(130, 1024)
(140, 1024)
(150, 1024)
(160, 1024)
(170, 1024)
(180, 1024)
(190, 1024)
(200, 1024)
(210, 1024)
(220, 1024)
(230, 1024)
(240, 1024)
(250, 1024)
(260, 1024)
(270, 1024)
(280, 1024)
(290, 1024)
(300, 1024)
(310, 1024)
(320, 1024)
(330, 1024)
(340, 1024)
(350, 1024)
(360, 1024)
(370, 1024)
(380, 1024)
(390, 1024)
(400, 1024)
(410, 1024)
(420, 1024)
(430, 1024)
(440, 1024)
(450, 1024)
(460, 1024)
(470, 1024)
(480, 1024)
(490, 1024)
(500, 1024)
(510, 1024)
(520, 1024)
(530, 1024)
(540, 1024)
(550, 1024)
(560, 1024)
(570, 1024)
(580, 1024)
(590, 1024)
(600, 1024)
(610, 1024)
(620, 1024)
(630, 1024)
(640, 1024)
(650, 1024)
(660, 1024)
(670, 1024)
(680, 1024)
(690, 1024)
(700, 1024)
(710, 1024)
(720, 1024)
(730, 1024)
(740, 1024)
(750, 1024)
(760, 1024)
(770, 1024)
(780, 1024)
(790, 1024)
(800, 1024)
(810, 1024)
(820, 1024)
(830, 1024)
(840, 1024)
(

In [30]:
df.to_csv("features_densenet.csv",index=False)

In [31]:
print(df.head())

   0     1     2         3         4         5     6         7         8     \
0   0.0   0.0   0.0  0.000449  0.048125  0.309170   0.0  0.000260  0.223262   
1   0.0   0.0   0.0  0.000479  0.053165  0.413816   0.0  0.000473  0.240986   
2   0.0   0.0   0.0  0.000279  0.023597  0.362767   0.0  0.000143  0.165157   
3   0.0   0.0   0.0  0.000291  0.025313  0.463563   0.0  0.000417  0.187508   
4   0.0   0.0   0.0  0.000707  0.029614  0.351612   0.0  0.000504  0.188743   

   9     ...      1014      1015      1016      1017      1018      1019  \
0   0.0  ...  0.450267  0.547136  2.287446  0.479416  0.451647  2.235839   
1   0.0  ...  0.433346  0.570787  1.880578  0.416098  0.656709  2.150825   
2   0.0  ...  0.213183  1.285135  1.673833  0.248113  0.992762  0.633769   
3   0.0  ...  0.532080  0.570855  2.361367  0.259937  0.441450  2.081491   
4   0.0  ...  0.212004  0.942717  2.264494  0.313245  0.802357  0.986290   

       1020      1021      1022      1023  
0  0.988726  2.863884  0